Referencia: <https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques>
____

# House Prices - Kaggle

Importamos Librerias

In [ ]:
import numpy as np
import pandas as pd
from vx.eda_vx import Eda, Graph
from vx.ml_vx import ML, Tools

Cargamos Datos

In [ ]:
df = pd.read_csv('train.csv', sep = ',', index_col= 'Id', na_values='NA')
df

Explicacion de los datos

Leer -> **``data_description.txt``**

### 1) Comenzamos analizando los datos

* Analizamos los nulos

In [ ]:
Eda.analizar_nulos(df)

* Hay muchos datos faltantes en algunas variables, lo que haremos es eliminar las variables que tengan mas del 50% de sus datos faltantes

In [ ]:
df = Eda.eliminar_nulos_si(df,p=0.5)
f'Nos quedan {df.shape[1]} variables'

* Ahora imputaremos los valores faltantes con el metodo de Mediana y Moda segun sea variable numerica o categorica respectivamente

In [ ]:
df = Eda.imputar_faltantes(df,metodo='mm')
Eda.analizar_nulos(df)

* Analicemos si tenemos variables con un solo valor y las eliminaremos

In [ ]:
df = Eda.eliminar_unitarios(df)
f'Quedan {df.shape[1]} variables'

* Analizemos graficamente las variables categoricas

In [ ]:
Graph.graficos_categoricos(df)

Nota: *Vemos algunas categorias bastante desbalanceadas, lo que puede influir en la precision de los resultados*

* Analicemos ahora las estadisticas numericas

In [ ]:
Eda.estadisticos_numericos(df)

Nota: *Vemos que tenemos amplio rango de valores en nuestras variables, lo que haremos sera estandarizarlas para que sus pesos no influyan en el resultado (usaremos Z-Score)*

In [ ]:
df = Eda.estandarizar_variables(df,"SalePrice",metodo="zscore")
df

* Ahora necesitaremos convertir a numericas nuestras variables categoricas, hay varias opciones pero utilizare dummies

In [ ]:
df = Eda.convertir_a_numericas(df,target="SalePrice",metodo="ohe")
df

Nota: *ahora tenemos 237 columnas, deberemos analizar si todas ellas aportan valor al modelo*

* Medimos el aporte de las variables mediante un modelo de Random Forest

In [ ]:
Tools.importancia_variables(df,"SalePrice",random_state=1,peores=15)

Nota: *Vemos que podemos eliminar varias variables que no nos aportan nada, por lo tanto vamos a eliminarlas*

In [ ]:
df = Tools.importancia_variables(df,"SalePrice",eliminar=True,umbral=0.00045,random_state=1)
df.shape[1]

### 2) Analicemos los modelos de Machine Learning posibles

* Correremos 3 modelos y elegiremos el que nos de el mejor resultado de ``mse``

In [ ]:
ML.modelo_catboost(df,"SalePrice","regresion",random_state=1)

In [ ]:
ML.modelo_lightgbm(df,"SalePrice","regresion",random_state=1)

In [ ]:
ML.modelo_xgboost(df,"SalePrice","regresion",random_state=1,save_model=True)

Nota: *El Modelo XGBoost es quien nos dio el menor error cuadratico medio (MSE) por lo que es el que utilizaremos*

* Trataremos de mejor nuestro modelo con una busqueda de hiperparametros activando el gridsearch

In [ ]:
ML.modelo_xgboost(df,"SalePrice","regresion",random_state=1,grid=True,save_model=True)

### 3) Ahora que tenemos el modelo entrenado y guardado, podemos usarlo para predecir los valores de las casas

* Cargamos los datos de test

In [ ]:
test = pd.read_csv('test.csv', sep = ',', index_col= 'Id', na_values='NA')
test

* Aplicamos los mismos cambios que hicimos al anterior set de datos

In [ ]:
# Hacemos una trampita para que no nos de problemas
test['A'] = np.random.choice(range(1, 100), size=1459)

In [ ]:
test = Eda.imputar_faltantes(test,metodo='mm')
test = Eda.convertir_a_numericas(test,target="A",metodo="ohe")
test

In [ ]:
# Lista de columnas en df
df1 = df.drop(columns=["SalePrice"],axis=1)
columns_to_keep = df1.columns.tolist()

# Elimina las columnas de test que no están en df
data = test[columns_to_keep]
data

* Cargamos el modelo y predecimos con los datos de test

In [ ]:
import joblib

# Ruta y nombre del archivo donde se guardó el modelo
model_filename = "xgboost.pkl"
# Cargar el modelo
loaded_model = joblib.load(model_filename)
# Ahora puedes utilizar el modelo cargado para hacer predicciones
# Supongamos que tienes un conjunto de datos 'X_test' para hacer predicciones
y_pred = loaded_model.predict(data)

* Mostramos los datos predecidos, agregamos los datos al dataset de test y guardamos en csv

In [ ]:
y_pred

In [ ]:
house_predict = pd.read_csv('test.csv', sep = ',', na_values='NA')
house_predict["SalePrice"] = y_pred

In [ ]:
try:
    house_predict.to_csv('house_predict.csv')
    print("Los datos se guardaron exitosamente")
except Exception as e:
    print(f"Se genero un problema durante el guardado. Descripcion {e}")

### 4) Datos a utilizar en Kaggle

In [ ]:
kaggle = house_predict[["Id","SalePrice"]]

In [ ]:
kaggle

In [ ]:
try:
    kaggle.to_csv('kaggle.csv',index=None)
    print("Los datos se guardaron exitosamente")
except Exception as e:
    print(f"Se genero un problema durante el guardado. Descripcion {e}")

### 5) Resultado Final en Kaggle con este modelo

In [ ]:
print(f'root-mean-square error (RMSE) = {0.14415} \nPosition: 1672/4101 \nBest R^2 (aqui) = 0.91')